In [2]:
from pyspark.sql import SparkSession,Row,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window



# Create SparkSession
spark=SparkSession.builder.master("local[1]")\
        .appName("Music")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mysql-connector-j-8.2.0.jar" ) \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
# print('PySpark Version :'+spark.version)
# print('PySpark Version :'+spark.sparkContext.version)

spark

24/05/26 22:15:50 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
24/05/26 22:15:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/26 22:15:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
tb_genres = spark.read.jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.tb_genres",properties={"user": "root", "password": "mysql"})
tb_genres.show()


+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+----------+
|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|          type|                 uri|          track_href|        analysis_url|duration_ms|time_signature|    genre|           song_name|     rowid|
+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+----------+
|        0.51|  0.86|  1|   -5.10|   1|       0.07|        0.00|            0.00|    0.40|   0.38| NULL|audio_features|spotify:track:11T...|https://api.spoti...|https://api.spoti...|     310160|             4|   trance|                NULL|8589

### MUSIC

In [3]:

df_music = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/data.csv')
df_music = df_music.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                               "regexp_replace(trim(artists),'[^a-zA-Z0-9]',' ') as artists",
                               "cast(danceability as decimal(3,2)) as danceability ",
                               "cast(energy as decimal(3,2)) as energy",
                               "duration_ms",
                               "explicit",
                               "id",
                               "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                               "key",
                               "cast (liveness as decimal(19,3) ) as liveness",
                               "cast(loudness as decimal(19,3)) as loudness",
                               "mode",
                               "name",
                               "popularity",
                               """ case when len(trim(release_date))<> 10 then '1990-01-10' else release_date
                                    end as release_date
                               """,
                               "cast(speechiness as decimal(5,2)) as speechiness",
                               "cast(tempo as decimal(19,3)) as tempo",
                               "cast(valence as decimal(3,2)) as valence",
                               "year",
                               "current_date() as dat_ref_carga"
                               )\
                   .withColumn('release_date',expr("case when release_date = 'Whipped into Shape' then '1990-01-10' else release_date end"))\
                   .withColumn('danceability', expr(" case when trim(regexp_replace(danceability,'[^a-zA-Z0-9]',' ')) = 'Michael Rupert' then 0 else danceability end  "))\
                   .withColumn("rowid", monotonically_increasing_id()) 

#df_music.filter("release_date = 'Whipped into Shape'").show(300,False)
df_music.show()

+------------+--------------------+------------+------+-----------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+-------------+-----+
|acousticness|             artists|danceability|energy|duration_ms|explicit|                  id|instrumentalness|key|liveness|loudness|mode|                name|popularity|release_date|speechiness|  tempo|valence|year|dat_ref_carga|rowid|
+------------+--------------------+------------+------+-----------+--------+--------------------+----------------+---+--------+--------+----+--------------------+----------+------------+-----------+-------+-------+----+-------------+-----+
|        1.00|   Carl Woitschach  |        0.71|  0.20|     158648|       0|6KbQ3uYMLKb5jDxLF...|            0.56| 10|   0.151| -12.428|   1|Singende Bataillo...|         0|  1990-01-10|       0.05|118.469|   0.78|1928|   2024-05-26|    0|
|        0.99|  Robert Schumann...|     

### tracks

In [ ]:
tracks = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/tracks.csv')

df_tracks = tracks.selectExpr("id"
                             ,"name"
                             ,"popularity"
                             ,"duration_ms"
                             ,"explicit"
                             ,"regexp_replace('artists','[^a-zA-Z0-9]',' ') as artists "
                             ,"regexp_replace('id_artists','[^a-zA-Z0-9]',' ') as id_artists"
                             ,"case when len(trim(release_date)) <> 10  then '1922-01-01' else release_date end as release_date"
                             ,"danceability"
                             ,"energy"
                             ,"key"
                             ,"loudness"
                             ,"mode"
                             ,"loudness"
                             ,"mode"
                             ,"speechiness"
                             ,"acousticness"
                             ,"instrumentalness"
                             ,"liveness"
                             ,"valence"
                             ,"tempo"
                             ,"time_signature")
df_tracks.show()

### GENRES

In [4]:
genres = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/genres_v2.csv')
genres = genres.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                  "cast(energy as decimal(3,2)) as energy",
                  "key",
                  "cast(loudness as decimal(3,2)) as loudness",
                  "mode",
                  "cast(speechiness as decimal(5,2)) as speechiness",
                  "cast(acousticness as decimal(3,2)) as acousticness",
                  "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                  "cast (liveness as decimal(3,2) ) as liveness",
                  "cast(valence as decimal(3,2)) as valence",
                  "cast(tempo as decimal(19,3)) as tempo",
                  "type",
                  "id",
                  "uri",
                  "track_href",
                  "analysis_url",
                  "duration_ms",
                  "time_signature",
                  "genre",
                  "song_name",
                  "title",
                  "current_date() as dat_ref_carga")\
        .withColumn("rowid", monotonically_increasing_id())

### artists

In [10]:
artists = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/artists.csv')
artists = artists.select("id","followers",expr("regexp_replace(genres,'[^a-zA-Z0-9]',' ') as genres"),"name","popularity")
artists = artists.withColumn("rank",expr("rank() over (partition by name order by name)"))
artists = artists.withColumn("row",expr("row_number() over (partition by name order by name)"))
artists = artists.withColumn("dt_hoje",expr("cast(date_format(current_date(), 'yyyyMMdd') as int)"))
artists = artists.withColumn("dt_adesao",expr("cast(date_format(current_date()-3, 'yyyyMMdd')as int)"))

# artistts = artists.filter("dt_adesao < dt_hoje")
artists.toPandas()


,id,followers,genres,name,popularity,rank,row,dt_hoje,dt_adesao
0,1r6nvWvnqJYcm8kW3cX1zQ,8187.0,australian children s music,"""""children's folk""""","""""children's music""""]""",1,1,20240526,20240523
1,7Aw6rjVbML8tamoAWm2Jpr,4972.0,canadian children s music,"""""children's folk""""","""""children's music""""]""",1,2,20240526,20240523
2,0UrDI2pvxUUaSgkaSkcpjb,8137.0,australian children s music,"""""children's folk""""","""""children's music""""]""",1,3,20240526,20240523
3,0SPvCvZvk7XDx8tGtGQhiE,10831.0,canadian children s music,"""""children's folk""""","""""children's music""""",1,4,20240526,20240523
4,5W9bmjm0mv5XitfBGynVN1,3853.0,canadian children s music,"""""children's folk""""","""""children's music""""",1,5,20240526,20240523
...,...,...,...,...,...,...,...,...,...
1162090,1gHh9Vs7eE9xDqiVXYGwvw,67.0,,황정음,0,1,1,20240526,20240523
1162091,4zTig4qoMQNeOZuIwUH03Z,0.0,,﻿The Mellstock Band and Singers,0,1,1,20240526,20240523
1162092,2wBpW4bAGYVe0yJcBeCTyd,55215.0,cyberpunk hardvapour vaporwave,２８１４,45,1,1,20240526,20240523
1162093,47bRmReD7vYEvvdzVMETDl,44.0,,ＴＯＫＩＯ２３,12,1,1,20240526,20240523


In [15]:
playlists = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/playlists.csv')
playlists = playlists.withColumn("rank",expr("rank() over (order by Genre)"))
playlists.show(truncate=False)


+----------------------+---------+----+
|Playlist              |Genre    |rank|
+----------------------+---------+----+
|19WuHd4MxWLzE1fpMmw4S4|Dark Trap|1   |
|6XyR8uzgkSoDzHuOxxRtLH|Dark Trap|1   |
|37Ij3ofyhvEhFEH8YZMZ2X|Dark Trap|1   |
|07zTlfPpsxeoWdumbkNWMI|Dark Trap|1   |
|2dClSRLsnptdkDQnpi5H2f|Dark Trap|1   |
|37i9dQZF1DX9wa6XirBPv8|Emo      |6   |
|6fmhLqO9CCMtOrJUeWasGO|Emo      |6   |
|7rxc8Bm5Gu8qbyJgH5ch2P|Emo      |6   |
|3Q97mNoNLZ4RyV9Css4zov|Emo      |6   |
|3vwjBackAZ0Rl9hueMkOwp|Emo      |6   |
|4zkArpBVbsHfR3JT4JW8Yc|Emo      |6   |
|4EH4sHWLCg1e3Et80vBQIP|Emo      |6   |
|5e8YeGZVploxholeRseSlR|Emo      |6   |
|5gs5kjYA88hfTx6X0KPdMo|Emo      |6   |
|7oe9PHYHNshF3oc3X0IIdy|Hiphop   |15  |
|1yiWwn8un5C1rIU77xdyQK|Hiphop   |15  |
|5byHCIUlXXjC62OTozi9Pb|Hiphop   |15  |
|3kxoamqBUb1k1uCvigVIJm|Hiphop   |15  |
|7jIWDTkV4dMkY4qg880NCZ|Hiphop   |15  |
|6mtYuOxzl58vSGnEDtZ9uB|Pop      |20  |
+----------------------+---------+----+
only showing top 20 rows

